In [5]:
import pandas as pd
import os
import cv2
import numpy as np
import math
import albumentations as A
import torch
import torch.optim as optim
from torch.utils.data import Dataset
from torch.utils.data import random_split, DataLoader
import torch.nn as nn
from torch.optim.lr_scheduler import ReduceLROnPlateau
import matplotlib.pyplot as plt
import torchvision.transforms.functional as TF
from PIL import Image
from torch.utils.data import ConcatDataset
import torch.nn.functional as F
from torchvision.ops import generalized_box_iou
import zipfile
import os
from google.colab import files
from sklearn.metrics import precision_recall_fscore_support


Saving Custom_Data.zip to Custom_Data.zip


In [10]:

image_dir = "/content/Exam Dataset/Custom_Data/Train/images"
label_dir = "/content/Exam Dataset/Custom_Data/Train/labels"
img_shape = (320,320)
RESIZE_SHAPE = (320, 320)

['images-14-_jpg.rf.95522ea2d2faa826141ddf33fc724738.jpg', 'good322_jpg.rf.b5bf949d7feeb66e629416291adb2d58.jpg', 'cheat267_jpg.rf.c931d4caf7c0692e9c92a848b9c3f1ec.jpg', 'good355_jpg.rf.7ac7b28c4a0f3ad0cbbd956045fa2183.jpg', 'good357_jpg.rf.11cbb16d7e5c1c62e58929189030c0e7.jpg', '20220616_140625_jpg.rf.74b9677d00f8e0cc942792d324f9fa4b.jpg', 'cheat1048_jpg.rf.24fae08cb875c3f89e842265726b44f7.jpg', 'good589_jpg.rf.4035fb7f5510d13ed762d949be60ad58.jpg', 'IMG_6415_jpg.rf.72970291b7e3389c07392e714c23f18f.jpg', '20220518_135537_jpg.rf.1c3211679b6949b0bc94ca9b1087135e.jpg', 'good143_jpg.rf.5dcd3773b5078728278c55084014882e.jpg', '20220616_140636_jpg.rf.e1514933f1a6ac66c5bfe691d894ece1.jpg', '13_JPG_jpg.rf.038bab9d3821e8bab146cac1cd1d9132.jpg', 'good580_jpg.rf.5023b170cd6c34b15730abfe3741b438.jpg', 'good321_jpg.rf.68093326c94b5ce16ab7eb3f5b64553d.jpg', 'youtube-349_jpg.rf.40015210f8280b18c39c2bcba8954f22.jpg', '20220622_185459_jpg.rf.97e05b39079e1a5aafb8be2a5e9082f3.jpg', 'good564_jpg.rf.2a1543

In [ ]:
Transform = A.Compose([
        A.HorizontalFlip(p=0.5),
        A.RandomRotate90(p=0.5),
        A.Transpose(p=0.2),
        A.RandomBrightnessContrast(p=0.5),
        A.HueSaturationValue(p=0.3),
        A.RGBShift(p=0.2),
        A.CLAHE(p=0.1),
        A.GaussNoise(std_range=(0.1, 0.2), p=0.5),
        A.ISONoise(p=0.2),
        A.MotionBlur(blur_limit=5, p=0.5),
        A.MedianBlur(blur_limit=3, p=0.4),
        A.GaussianBlur(blur_limit=(3, 5), p=0.5),
        A.RandomScale(scale_limit=0.1, p=0.3),
        A.ShiftScaleRotate(shift_limit=0.05, scale_limit=0.1, rotate_limit=10, border_mode=0, p=0.4),

        # Adaptive padding and cropping
        A.PadIfNeeded(min_height=320, min_width=320, border_mode=0),
        A.RandomCrop(height=320, width=320, p=0.8),
        A.Resize(height=320, width=320, p=1.0)
    ], bbox_params=A.BboxParams(format='pascal_voc', label_fields=['class_labels']))

resize_transform = A.Compose([  A.PadIfNeeded(min_height=320, min_width=320, border_mode=0),
        A.RandomCrop(height=320, width=320, p=0.8),
        A.Resize(height=320, width=320, p=1.0)],bbox_params=A.BboxParams(format='pascal_voc', label_fields=['class_labels']))

In [ ]:
class YOLODataset(Dataset):
    def __init__(self, images_dir, labels_dir, transform=None):
        self.images_dir = images_dir
        self.labels_dir = labels_dir
        self.transform = transform
        self.image_files = [f for f in os.listdir(images_dir) if f.endswith(('.jpg', '.png'))]

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        # Load image
        img_filename = self.image_files[idx]
        img_path = os.path.join(self.images_dir, img_filename)
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        original_height, original_width, _ = image.shape

        # Load labels
        label_path = os.path.join(self.labels_dir, os.path.splitext(img_filename)[0] + ".txt")
        bboxes, class_labels = [], []

        if os.path.exists(label_path):
            with open(label_path, 'r') as f:
                for line in f:
                    class_id, x_c, y_c, w, h = map(float, line.strip().split())
                    x_min = max((x_c - w/2) * original_width, 0)
                    y_min = max((y_c - h/2) * original_height, 0)
                    x_max = min((x_c + w/2) * original_width, original_width - 1)
                    y_max = min((y_c + h/2) * original_height, original_height - 1)
                    # Filter invalid boxes
                    if x_max > x_min and y_max > y_min:
                        bboxes.append([x_min, y_min, x_max, y_max])
                        class_labels.append(int(class_id))

        # Apply transforms if available
        if self.transform and len(bboxes) > 0:
            try:
                transformed = self.transform(image=image, bboxes=bboxes, class_labels=class_labels)
                image = transformed['image']
                bboxes = transformed['bboxes']
                class_labels = transformed['class_labels']
            except ValueError as e:
                valid_bboxes, valid_labels = [], []
                for bbox, label in zip(bboxes, class_labels):
                    x_min, y_min, x_max, y_max = bbox
                    if x_max > x_min and y_max > y_min:
                        valid_bboxes.append(bbox)
                        valid_labels.append(label)
                if len(valid_bboxes) > 0:
                    try:
                        transformed = self.transform(image=image, bboxes=valid_bboxes, class_labels=valid_labels)
                        image = transformed['image']
                        bboxes = transformed['bboxes']
                        class_labels = transformed['class_labels']
                    except ValueError:

                        pass


        image = image / 255.0
        image = np.transpose(image, (2, 0, 1))  # HWC → CHW
        image = torch.tensor(image, dtype=torch.float32)


        current_height, current_width = image.shape[1], image.shape[2]
        image = TF.resize(image, RESIZE_SHAPE)
        new_height, new_width = RESIZE_SHAPE


        scale_x = new_width / current_width
        scale_y = new_height / current_height


        scaled_bboxes = []
        for bbox in bboxes:
            x_min, y_min, x_max, y_max = bbox
            scaled_x_min = x_min * scale_x
            scaled_y_min = y_min * scale_y
            scaled_x_max = x_max * scale_x
            scaled_y_max = y_max * scale_y

            # Clamp to image boundaries
            scaled_x_min = max(0, min(scaled_x_min, new_width - 1))
            scaled_y_min = max(0, min(scaled_y_min, new_height - 1))
            scaled_x_max = max(0, min(scaled_x_max, new_width - 1))
            scaled_y_max = max(0, min(scaled_y_max, new_height - 1))


            if scaled_x_max > scaled_x_min and scaled_y_max > scaled_y_min:
                scaled_bboxes.append([scaled_x_min, scaled_y_min, scaled_x_max, scaled_y_max])
            else:

                class_labels.pop(len(scaled_bboxes))


        targets = []
        for box, label in zip(scaled_bboxes, class_labels):
            targets.append([label] + list(box))
        targets = torch.tensor(targets, dtype=torch.float32) if targets else torch.zeros((0, 5), dtype=torch.float32)

        return image, targets

In [ ]:
class ConcatDatasets(Dataset):
    def __init__(self, dataset1, dataset2):
        self.dataset1 = dataset1
        self.dataset2 = dataset2

    def __len__(self):
        return len(self.dataset1) + len(self.dataset2)

    def __getitem__(self, idx):
        if idx < len(self.dataset1):
            return self.dataset1[idx]
        else:
            return self.dataset2[idx - len(self.dataset1)]

In [ ]:

base_dataset = YOLODataset(
    images_dir=image_dir,
    labels_dir=label_dir,
)

transformed_dataset = YOLODataset(
    images_dir=image_dir,
    labels_dir=label_dir,
    transform=Transform
)

doubled_dataset = ConcatDatasets(base_dataset, transformed_dataset)

print("The number of images in this Doubled dataset is ",len(doubled_dataset))
train_size = int(0.8 * len(doubled_dataset))
val_size = len(doubled_dataset) - train_size
def collate_fn(batch):
    images, targets = zip(*batch)
    images = torch.stack(images)
    return images, list(targets)

train_dataset, val_dataset = random_split(doubled_dataset, [train_size, val_size])
dataloader = DataLoader(doubled_dataset, batch_size=4, shuffle=True, collate_fn=collate_fn)
train_loader = DataLoader(
    train_dataset,
    batch_size=64,
    shuffle=True,
    collate_fn=collate_fn
)

val_loader = DataLoader(
    val_dataset,
    batch_size=64,
    shuffle=False,
    collate_fn=collate_fn
)

The number of images in this Doubled dataset is  9764


In [ ]:

class DetectionLoss(nn.Module):
    def __init__(self):
        super(DetectionLoss, self).__init__()
        self.class_loss_fn = nn.BCEWithLogitsLoss()
        self.bbox_loss_fn = nn.SmoothL1Loss()

    def forward(self, predictions, targets):
        """
        predictions: [B, K, 5]
        targets: list of [N_i, 5] tensors
        """
        B, K, _ = predictions.shape
        pred_scores = predictions[:, :, 0]       # [B, K]
        pred_boxes = predictions[:, :, 1:5]      # [B, K, 4]

        true_labels = torch.zeros((B, K), device=predictions.device)
        true_boxes = torch.zeros((B, K, 4), device=predictions.device)

        for i, t in enumerate(targets):
            num_objs = min(K, t.shape[0])
            if num_objs > 0:
                true_labels[i, :num_objs] = t[:num_objs, 0]
                true_boxes[i, :num_objs] = t[:num_objs, 1:5]

        class_loss = self.class_loss_fn(pred_scores, true_labels)
        bbox_loss = self.bbox_loss_fn(pred_boxes, true_boxes)

        return class_loss + bbox_loss

class EarlyStopping:
    def __init__(self, patience=5, restore_best_weights=True):
        self.patience = patience
        self.counter = 0
        self.best_loss = float('inf')
        self.best_model_state = None
        self.restore_best_weights = restore_best_weights
        self.early_stop = False

    def __call__(self, val_loss, model):
        if val_loss < self.best_loss:
            self.best_loss = val_loss
            self.counter = 0
            if self.restore_best_weights:
                self.best_model_state = {k: v.cpu() for k, v in model.state_dict().items()}
        else:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True
                if self.restore_best_weights and self.best_model_state is not None:
                    model.load_state_dict(self.best_model_state)

def save_checkpoint(model, epoch, val_metric, best_metric, path='best_model.pth'):
    if val_metric > best_metric:
        print(f"Validation metric improved ({best_metric:.4f} → {val_metric:.4f}). Saving model.")
        torch.save(model.state_dict(), path)
        return val_metric
    return best_metric

class ObjectDetectionCNN(nn.Module):
    def __init__(self, input_channels=3, num_predictions=2):
        super(ObjectDetectionCNN, self).__init__()

        def conv_block(in_channels, out_channels, num_convs, pool=True):
            layers = []
            for _ in range(num_convs):
                layers.append(nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1))
                layers.append(nn.ReLU(inplace=True))
                in_channels = out_channels
            if pool:
                layers.append(nn.MaxPool2d(kernel_size=2, stride=2))
            return nn.Sequential(*layers)

        self.features = nn.Sequential(
            conv_block(input_channels, 64, num_convs=2),
            conv_block(64, 128, num_convs=2),
            conv_block(128, 256, num_convs=4),
            conv_block(256, 512, num_convs=4),
            conv_block(512, 512, num_convs=4),
        )

        self.adapt_pool = nn.AdaptiveAvgPool2d((7, 7))
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(512 * 7 * 7, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, num_predictions * 5)
        )
        self.num_predictions = num_predictions

    def forward(self, x):
        x = self.features(x)
        x = self.adapt_pool(x)
        x = self.classifier(x)
        x = x.view(x.shape[0], self.num_predictions, 5)
        return x

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = ObjectDetectionCNN().to(device)
num_epochs = 25
optimizer = optim.Adam(model.parameters(), lr=1e-4)
criterion = DetectionLoss()
early_stopping = EarlyStopping(patience=2, restore_best_weights=True)
best_val_loss = float('inf')
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=2, min_lr=1e-6)

for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0

    for batch_images, batch_targets in dataloader:
        batch_images = batch_images.to(device)
        batch_targets = [t.to(device) for t in batch_targets]
        outputs = model(batch_images)
        loss = criterion(outputs, batch_targets)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_loss += loss.item()

    model.eval()
    val_loss = 0.0

    with torch.no_grad():
        for val_images, val_targets in val_loader:
            val_images = val_images.to(device)
            val_targets = [t.to(device) for t in val_targets]

            outputs = model(val_images)
            loss = criterion(outputs, val_targets)
            val_loss += loss.item()

    avg_train_loss = train_loss / len(dataloader)
    avg_val_loss = val_loss / len(val_loader)

    print(f"Epoch {epoch+1}: Train Loss = {avg_train_loss:.4f}, Val Loss = {avg_val_loss:.4f}")

    # Early stopping
    early_stopping(avg_val_loss, model)
    if early_stopping.early_stop:
        print("Early stopping triggered.")
        break

    # Reduce LR on plateau
    scheduler.step(avg_val_loss)

Epoch 1: Train Loss = 64.5541, Val Loss = 63.2544
Epoch 2: Train Loss = 63.5297, Val Loss = 64.1287
Epoch 3: Train Loss = 63.2917, Val Loss = 63.3330
Early stopping triggered.


In [ ]:
def draw_boxes(image, boxes, labels, color=(0, 255, 0), label_prefix=""):
    """Draw bounding boxes and labels on image"""
    image = image.copy()
    for box, label in zip(boxes, labels):
        x1, y1, x2, y2 = map(int, box)
        cv2.rectangle(image, (x1, y1), (x2, y2), color, 2)
        cv2.putText(image, f"{label_prefix}{label}", (x1, y1 - 5),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1)
    return image

def add_prediction_status(image, is_correct, position='top'):
    """Add prediction status text to image"""
    image = image.copy()
    status_text = "CORRECT PREDICTION" if is_correct else "INCORRECT PREDICTION"
    color = (0, 255, 0) if is_correct else (0, 0, 255)


    font = cv2.FONT_HERSHEY_SIMPLEX
    font_scale = 0.8
    thickness = 2
    (text_width, text_height), baseline = cv2.getTextSize(status_text, font, font_scale, thickness)

    if position == 'top':
        x = (image.shape[1] - text_width) // 2
        y = text_height + 10
    else:
        x = (image.shape[1] - text_width) // 2
        y = image.shape[0] - 10


    cv2.rectangle(image, (x-5, y-text_height-5), (x+text_width+5, y+baseline+5), (255, 255, 255), -1)
    cv2.putText(image, status_text, (x, y), font, font_scale, color, thickness)

    return image

import os
import torch
import numpy as np
import cv2
from sklearn.metrics import precision_recall_fscore_support

def evaluate_model_binary(model, test_loader, device, threshold=0.5, save_dir="pred_vs_true", max_images=100):
    os.makedirs(save_dir, exist_ok=True)
    model.eval()

    y_true, y_pred = []
    saved_count = 0

    print(f"Evaluating... (threshold={threshold})")

    with torch.no_grad():
        for batch_idx, (images, targets) in enumerate(test_loader):
            images = images.to(device)
            outputs = model(images)

            for i, (output, target) in enumerate(zip(outputs, targets)):

                if isinstance(target, torch.Tensor):
                    target = target.to(device)
                    true_class = 1 if (target.ndim >= 1 and target[0] > 0.5) else 0
                else:
                    continue


                pred_objectness = torch.sigmoid(output[0, 0]).item()
                pred_class = 1 if pred_objectness > threshold else 0

                y_true.append(true_class)
                y_pred.append(pred_class)


                if saved_count < max_images:
                    img_np = images[i].cpu().numpy().transpose(1, 2, 0) * 255
                    img_np = img_np.astype(np.uint8)
                    if img_np.shape[2] == 3:
                        img_np = cv2.cvtColor(img_np, cv2.COLOR_RGB2BGR)


                    pred_boxes, pred_labels = [], []
                    for pred in output:
                        score = torch.sigmoid(pred[0]).item()
                        if score > threshold:
                            pred_boxes.append(pred[1:5].tolist())
                            pred_labels.append(f"P:{score:.2f}")


                    true_boxes, true_labels = [], []
                    if target.ndim == 2:
                        for t in target:
                            if len(t) >= 5:
                                _, x1, y1, x2, y2 = t.tolist()
                                true_boxes.append([x1, y1, x2, y2])
                                true_labels.append("GT")

                    pred_img = draw_boxes(img_np.copy(), pred_boxes, pred_labels, (0, 0, 255), "P:")
                    true_img = draw_boxes(img_np.copy(), true_boxes, true_labels, (0, 255, 0), "T:")

                    # Add comparison metadata
                    info_text = f"Pred: {pred_class} | True: {true_class} | Score: {pred_objectness:.2f}"
                    for img in [pred_img, true_img]:
                        cv2.putText(img, info_text, (10, img.shape[0] - 10),
                                    cv2.FONT_HERSHEY_SIMPLEX, 0.4, (255, 255, 255), 1)

                    label_strip = np.zeros((30, img_np.shape[1], 3), dtype=np.uint8)
                    cv2.putText(label_strip, "GROUND TRUTH", (10, 20),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
                    true_combined = np.vstack([label_strip, true_img])

                    label_strip_pred = label_strip.copy()
                    cv2.putText(label_strip_pred, "PREDICTION", (10, 20),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
                    pred_combined = np.vstack([label_strip_pred, pred_img])

                    combined_img = np.concatenate([true_combined, pred_combined], axis=1)
                    cv2.imwrite(os.path.join(save_dir, f"sample_{saved_count:03d}.jpg"), combined_img)
                    saved_count += 1

            if (batch_idx + 1) % 5 == 0:
                acc = 100 * np.mean(np.array(y_true) == np.array(y_pred))
                print(f"Processed {batch_idx + 1} batches - Accuracy so far: {acc:.2f}%")


    precision, recall, f1, _ = precision_recall_fscore_support(y_true, y_pred, average='binary', zero_division=0)
    accuracy = 100 * np.mean(np.array(y_true) == np.array(y_pred))

    print("\n" + "="*60)
    print(f"Final Accuracy: {accuracy:.2f}%")
    print(f"Recall: {recall:.2f}")
    print(f"F1 Score: {f1:.2f}")
    print(f"Images saved to: {save_dir}")
    print("="*60)

    return {
        "accuracy": accuracy,
        "recall": recall,
        "f1_score": f1,
        "saved_images": saved_count
    }

def load_model(model_path, device):
    """Load a saved model"""

    model = ObjectDetectionCNN().to(device)


    checkpoint = torch.load(model_path, map_location=device)
    model.load_state_dict(checkpoint['model_state_dict'])
    model.eval()

    print(f"Model loaded from: {model_path}")
    print(f"Training was stopped at epoch: {checkpoint['epoch']}")
    return model

In [ ]:
from datetime import datetime
os.makedirs('saved_models', exist_ok=True)
model_path = f'saved_models/object_detection_model_{datetime.now().strftime("%Y%m%d_%H%M%S")}.pth'
torch.save({
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'epoch': epoch,
    'train_loss': avg_train_loss,
    'val_loss': avg_val_loss,
    'model_architecture': 'ObjectDetectionCNN'
}, model_path)

print(f"Model saved to: {model_path}")

Model saved to: saved_models/object_detection_model_20250609_185403.pth


In [ ]:
test_image_dir = "../input/exam-cheating-detection/4000Final/test/images"
test_label_dir = "../input/exam-cheating-detection/4000Final/test/labels"
test_transform = A.Compose([A.Resize(height=320, width=320, p=1.0)])
test_dataset = YOLODataset(
    images_dir=test_image_dir,
    labels_dir=test_label_dir,
    transform = test_transform
)

test_loader = DataLoader(
    test_dataset,
    batch_size=64,
    shuffle=True,
    collate_fn=collate_fn
)
model = load_model('/kaggle/working/saved_models/object_detection_model_20250609_185403.pth',device)
simple_accuracy_check(model,test_loader,device)

Model loaded from: /kaggle/working/saved_models/object_detection_model_20250609_185403.pth
Training was stopped at epoch: 2
Starting accuracy check with threshold: 0.5
Results will be saved to:
  - Correct predictions: pred_vs_true/correct_predictions
  - Incorrect predictions: pred_vs_true/incorrect_predictions
Processed 5 batches, Current accuracy: 60.62%

Final Objectness Accuracy: 62.70% (311/496)
Correct predictions: 311 samples
Incorrect predictions: 185 samples
Images saved:
  - Correct predictions: 25 images in pred_vs_true/correct_predictions
  - Incorrect predictions: 25 images in pred_vs_true/incorrect_predictions


62.70161290322581

In [ ]:
import matplotlib.image as mpimg
j=1
for i in range(1, 20):
    img_path = f'/kaggle/working/pred_vs_true/correct_predictions/correct_000_batch0_img{i}.jpg'

    # Load and display the image
    img = mpimg.imread(img_path)
    plt.imshow(img)
    plt.axis('off')  # Hide axis
    plt.title(f'Image {i:03d}')
    plt.show()